In [3]:
import pandas as pd
import numpy as np
import math

# Exercise 1

In [4]:
# This code is provided for you. Please read it and understand what it's doing
def payment_df_to_cohort_df(payment_df):
    # Calculate the cohort month for each customer. This is the first month that the customer made a payment
    cohort_df = payment_df.groupby('customer_id').payment_date.min().reset_index()
    cohort_df['cohort'] = cohort_df['payment_date'].dt.to_period('M').dt.to_timestamp()
    payment_df = payment_df.merge(cohort_df[['customer_id', 'cohort']], on='customer_id')
    # Round payments to the nearest month. month needs to be a pd.datetime still
    # Using payment_month simplifies how CVF thinks about cohorts
    payment_df['payment_month'] = payment_df['payment_date'].dt.to_period('M').dt.to_timestamp()
    # Calculate the payment period. However, I want the result to be an integer month
    # payment_period integers are one way to understand cohorts
    payment_df['payment_period'] = (
        (payment_df['payment_month'].dt.year - payment_df['cohort'].dt.year) * 12 +
        (payment_df['payment_month'].dt.month - payment_df['cohort'].dt.month)
    )
    grouped_df = payment_df.groupby(["cohort", "payment_period"])["amount"].sum()
    grouped_df = grouped_df.reset_index().pivot(index="cohort", columns="payment_period", values="amount").rename_axis(index=None, columns=None)
    return grouped_df




In [5]:
# test_payment_df = pd.DataFrame({
#     'customer_id': [1, 1, 2, 2, 3, 3, 3],
#     'payment_date': pd.to_datetime(['2020-01-01', '2020-02-01', '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01']),
#     'amount': [200, 100, 400, 300, 500, 600, 700]
# })



test_payment_df = pd.DataFrame({
    'customer_id': [1, 1, 2, 2, 3],
    'payment_date': pd.to_datetime(['2020-01-01', '2020-02-01', '2020-02-01', '2020-03-01', '2020-04-01']),
    'amount': [200, 100, 400, 300, 500]
})






In [6]:
EXERCISE_1_COHORT_DF = payment_df_to_cohort_df(test_payment_df)
EXERCISE_1_COHORT_DF

,0,1
2020-01-01,200.0,100.0
2020-02-01,400.0,300.0
2020-04-01,500.0,NaN


In [7]:
# Example spend file. Each cohort is spending $1000 a month
# We don't actually use the spend_df until exercise 3
EXERCISE_SPEND_DF = pd.DataFrame({ 
    "cohort_start_at": [pd.Timestamp(date) for date in ['2020-01-01 00:00:00', '2020-02-01 00:00:00', '2020-04-01 00:00:00']],
    "spend": [1000.0, 1000.0, 1000.0],
}).set_index("cohort_start_at")
EXERCISE_SPEND_DF

,spend
cohort_start_at,
2020-01-01,1000.0
2020-02-01,1000.0
2020-04-01,1000.0


# Exercise 2

In [8]:
def apply_predictions_list(m0 : float, churn : float, actual_values : list = list[float], n_months : int = 6) -> list:
    
    # m0 is the initial predicted value of the cohort payment for month 0
    # if actual_values are provided, then use the last actual value as the initial predicted value
    m0 = actual_values[-1] if actual_values else m0
    new_vals = [round(m0 * pow((1-churn),i), 2) for i in range(n_months - len(actual_values))]
    return actual_values + new_vals

In [9]:
def apply_predictions_to_cohort_df(predictions_dict : dict, cohort_df :pd.DataFrame) -> pd.DataFrame:
    churn = predictions_dict['churn']
    TOT_COLS = 12
    df = cohort_df.copy()

    for i in range(df.columns[-1]+1, TOT_COLS):
        df[i] = np.nan

    n_rows, n_cols = df.shape

    for r in range(n_rows):
        
        row_vals = df.iloc[r].to_numpy(dtype=float)

        notna = ~np.isnan(row_vals)
        if not notna.any():
            continue  # nothing to base decay on

        last_pos = n_cols - 1 - np.argmax(notna[::-1])
        last_val = row_vals[last_pos]

        k = 1
        for c in range(last_pos + 1, n_cols):
            df.iloc[r, c] = last_val * (1 - churn) ** k
            k += 1
    
    return df
    

In [10]:
# We will extend the cohort dataframe from exercise 1
EXERCISE_2_BEST_PREDICTIONS_DICT = {
        "m0": 0.4,
        "churn": 0.1,
}
# Showing how predictions look different per scenario
EXERCISE_2_AVERAGE_PREDICTIONS_DICT = {
        "m0": 0.3,
        "churn": 0.2,
}
EXERCISE_2_WORST_PREDICTIONS_DICT = {
        "m0": 0.1,
        "churn": 0.3,
}

In [11]:
# Compare your results to these expected values
"""
                0      1      2      3       4       5
2020-01-01  200.0  100.0   90.0   81.0   72.90   65.61
2020-02-01  400.0  300.0  270.0  243.0  218.70  196.83
2020-04-01  500.0  450.0  405.0  364.5  328.05  295.25
"""
EXPECTED_PREDICTED_SERIES = [0.4, 0.36, 0.32, 0.29, 0.26, 0.24, 0.21, 0.19, 0.17, 0.15, 0.14, 0.13]
assert apply_predictions_list(EXERCISE_2_BEST_PREDICTIONS_DICT["m0"], EXERCISE_2_BEST_PREDICTIONS_DICT["churn"], [], 12) == EXPECTED_PREDICTED_SERIES

apply_predictions_to_cohort_df(EXERCISE_2_BEST_PREDICTIONS_DICT, EXERCISE_1_COHORT_DF)


,0,1,2,3,4,5,6,7,8,9,10,11
2020-01-01,200.0,100.0,90.0,81.0,72.90,65.610,59.0490,53.14410,47.829690,43.046721,38.742049,34.867844
2020-02-01,400.0,300.0,270.0,243.0,218.70,196.830,177.1470,159.43230,143.489070,129.140163,116.226147,104.603532
2020-04-01,500.0,450.0,405.0,364.5,328.05,295.245,265.7205,239.14845,215.233605,193.710245,174.339220,156.905298


# Exercise 3

In [526]:
def apply_threshold_to_cohort_df(cohort_df : pd.DataFrame, spend_df : pd.DataFrame, threshold : list[dict]) -> pd.DataFrame:
    
    n_rows, n_cols = cohort_df.shape
    df = pd.DataFrame(True, index = cohort_df.index, columns=cohort_df.columns, dtype=bool)

    for th in threshold:
        payment_period_month = th['payment_period_month']
        min_payment_percent = th['minimum_payment_percent']
        
        for i in range(n_rows):
            df.iloc[i, payment_period_month] = (cohort_df.iloc[i, payment_period_month] / spend_df.iloc[i, 0]) > min_payment_percent
    
    return df

In [527]:
EXAMPLE_THRESHOLD_DICT = [{
    "payment_period_month": 1,
    "minimum_payment_percent": 0.25
}]

In [528]:
""" The above should return a threshold_df that looks like this
                0     1
2020-01-01   True  False
2020-02-01   True  True
2020-04-01   True  False
"""
threshold_df = apply_threshold_to_cohort_df(EXERCISE_1_COHORT_DF, EXERCISE_SPEND_DF, EXAMPLE_THRESHOLD_DICT)
display(threshold_df)


,0,1
2020-01-01,True,False
2020-02-01,True,True
2020-04-01,True,False


# Exercise 4

In [570]:
def process_cohort(payments: pd.Series, spend: int, sharing_percentage: float, cash_cap: int, threshold : list[dict]) -> pd.Series:
    threshold_dict = {th["payment_period_month"]: th["minimum_payment_percent"] for th in threshold}

    
    tot = 0
    collection = []
    for i in range(len(payments)):
        threshold = threshold_dict.get(i, None)
        val = payments[i]
        sharing = 1 if threshold and (val / spend) <= threshold else sharing_percentage

        to_collect = min(sharing * val, cash_cap - tot)
        tot += to_collect
        collection.append(to_collect)

        if tot > cash_cap:
            raise ValueError("Total >= cash_cap")
        elif tot == cash_cap:
            break

    return collection

def get_cvf_cashflows_df(cohort_df : pd.DataFrame, spend_df : pd.DataFrame, threshold : list[dict], trade_list : dict) -> pd.DataFrame:
    # Go through each trade in trade_list and apply the correct sharing_percentage per cohort
    # Apply the cash_cap to the cohort_df (cut off after the sum reaches the cash_cap)
    # Apply the payment_delay (add months to the payment_periods)

    cols = pd.date_range(start=cohort_df.index[0], periods=15, freq="MS")
    
    df = pd.DataFrame(0, cohort_df.index, cols, dtype='float')
    
    for trade in trade_list:
        timestamp, sharing_percentage, cash_cap = trade['cohort_start'], trade['sharing_percentage'], trade['cash_cap']
        payments = cohort_df.loc[timestamp]
        spend = spend_df.loc[timestamp, 'spend']

        collection_series = process_cohort(payments, spend, sharing_percentage, cash_cap, threshold)
        i = df.index.get_loc(timestamp)
        j = df.columns.get_loc(timestamp)

        df.iloc[i, j:j+len(collection_series)] = collection_series

    return df.fillna(0)

In [571]:
"""This represents the final output of the function when run only on the cohort_df.

            2020-01-01  2020-02-01  2020-03-01  2020-04-01  2020-05-01
# We have not reached the cash_cap, but we are collecting 100% payments in month 1
2020-01-01       160.0       100.0         0.0         0.0         0.0
# We have reached the cash_cap
2020-02-01         0.0       320.0        80.0         0.0         0.0
# We have not reached the cash_cap
2020-04-01         0.0         0.0         0.0       250.0         0.0
"""
EXAMPLE_TRADE_LIST_1 = [{
    "cohort_start": pd.Timestamp('2020-01-01 00:00:00'),
    "sharing_percentage": 0.8,
    "cash_cap": 500,
},
{
    "cohort_start": pd.Timestamp('2020-02-01 00:00:00'),
    "sharing_percentage": 0.8,
    "cash_cap": 400,
},
{
    "cohort_start": pd.Timestamp('2020-04-01 00:00:00'),
    "sharing_percentage": 0.5,
    "cash_cap": 500,
}]



cvf_cashflows_df = get_cvf_cashflows_df(EXERCISE_1_COHORT_DF, EXERCISE_SPEND_DF, EXAMPLE_THRESHOLD_DICT, EXAMPLE_TRADE_LIST_1)
cvf_cashflows_df


,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01
2020-01-01,160.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-01,0.0,320.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-04-01,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [572]:

"""This represents the final output of the function when run on the best predicted cohort_df

            2020-01-01  2020-02-01  2020-03-01  2020-04-01  2020-05-01  \
# We do not hit our cash cap, even after collecting 100% payments in month 0
2020-01-01       200.0        80.0        90.0        64.8       58.32
# We hit our cash cap, after collecting 100% payments in month 2
2020-02-01         0.0       320.0       240.0       270.0      194.40
# We hit our cash cap, without collection any payments at 100%
2020-04-01         0.0         0.0         0.0       250.0      225.00

            2020-06-01  2020-07-01  2020-08-01  2020-09-01  2020-10-01  \
2020-01-01       52.49       47.24       42.51       38.26       34.44
2020-02-01       75.60        0.00        0.00        0.00        0.00
2020-04-01      202.50      182.25      164.02       76.23        0.00

            2020-11-01  2020-12-01  2021-01-01  2021-02-01  2021-03-01
2020-01-01       30.99        27.9         0.0         0.0         0.0
2020-02-01        0.00         0.0         0.0         0.0         0.0
2020-04-01        0.00         0.0         0.0         0.0         0.0
"""
EXAMPLE_TRADE_LIST_2 = [{
    "cohort_start": pd.Timestamp('2020-01-01 00:00:00'),
    "sharing_percentage": 0.8,
    "cash_cap": 1100,
},
{
    "cohort_start": pd.Timestamp('2020-02-01 00:00:00'),
    "sharing_percentage": 0.8,
    "cash_cap": 1100,
},
{
    "cohort_start": pd.Timestamp('2020-04-01 00:00:00'),
    "sharing_percentage": 0.5,
    "cash_cap": 1100,
}]
EXAMPLE_THRESHOLD_DICT_2 = [
    {
        "payment_period_month": 0,
        "minimum_payment_percent": 0.3
    },
    {
        "payment_period_month": 2,
        "minimum_payment_percent": 0.3
    },
]

predicted_cohort_df = apply_predictions_to_cohort_df(EXERCISE_2_BEST_PREDICTIONS_DICT, EXERCISE_1_COHORT_DF)
predicted_cvf_cashflows_df = get_cvf_cashflows_df(predicted_cohort_df, EXERCISE_SPEND_DF, EXAMPLE_THRESHOLD_DICT_2, EXAMPLE_TRADE_LIST_2)
predicted_cvf_cashflows_df

,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01
2020-01-01,200.0,80.0,90.0,64.8,58.32,52.488,47.2392,42.51528,38.263752,34.437377,30.993639,27.894275,0.0,0.0,0.0
2020-02-01,0.0,320.0,240.0,270.0,194.40,75.600,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2020-04-01,0.0,0.0,0.0,250.0,225.00,202.500,182.2500,164.02500,76.225000,0.000000,0.000000,0.000000,0.0,0.0,0.0


Components of DB

First DB table should contain all of the raw transaction data given to us by the companies

Functions
- edit the raw transaction data
- 